# Read WFS data from USGS ScienceBase into Shapely/Cartopy

In [4]:
!pip show pyproj

Name: pyproj
Version: 1.9.5.1
Summary: Python interface to PROJ.4 library
Home-page: https://github.com/jswhit/pyproj
Author: Jeff Whitaker
Author-email: jeffrey.s.whitaker@noaa.gov
License: OSI Approved
Location: /home/zeppelin-v/.local/lib/python3.5/site-packages
Requires: 
Required-by: OWSLib


Based on notebook by Rich Signell. Use to test connection to GeoServer WFS with JSON output format

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import geojson
from owslib.wfs import WebFeatureService
from shapely.geometry import Polygon, Point, mapping, asShape, shape
# import cartopy.crs as ccrs
# from cartopy.io.img_tiles import MapQuestOpenAerial, MapQuestOSM, OSM
%matplotlib inline

In [18]:
# Read shapefile data from USGS ScienceBase WFS 1.1 service in JSON format
# (shapefile was previosly uploaded to ScienceBase, creating the WFS service)

In [19]:
# getCapabilities
#https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c54be4b0aa151574a8dc?service=wfs&version=1.0.0&request=GetCapabilities
# containes LatLongBoundingBox for each feature

In [20]:
# some USGS ScienceBase Geoserver WFS endpoints:
#endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c54be4b0aa151574a8dc'
endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c5fce4b0aa151574a8ed'
#endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342e124e4b0aa151574a969'
wfs = WebFeatureService(endpoint, version='1.1.0')
print ("wfs version",wfs.version)

wfs version 1.1.0


In [21]:
shp = wfs.contents.keys()
print(shp)

dict_keys(['sb:Conservation_Zone_WGS84', 'sb:footprint'])


In [22]:
a = wfs.contents['sb:footprint']
b = a.boundingBoxWGS84

In [23]:
shp = filter(lambda a: a != 'sb:footprint', shp)
featurelist=list(shp)
print(featurelist)


['sb:Conservation_Zone_WGS84']


In [24]:
def flip_geojson_coordinates(geo):
    if isinstance(geo, dict):
        for k in geo:
            #print("key: ",k)
            if k == "coordinates":
                z = np.asarray(geo[k])
                f = z.flatten()
                geo[k] = np.dstack((f[1::2], f[::2])).reshape(z.shape).tolist()
            else:
                flip_geojson_coordinates(geo[k])
    elif isinstance(geo, list):
        for k in geo:
            #print('list key {0}'.format(k))
            flip_geojson_coordinates(k)

In [25]:
#srs='EPSG:4326' # v1.0 syntax
srs='urn:x-ogc:def:crs:EPSG:4326'  # v1.1 syntax
json_response = wfs.getfeature(typename=[featurelist[0]], propertyname=None, srsname=srs, outputFormat='application/json').read()
#json_response = wfs.getfeature(typename=[next(shp)], propertyname=None, srsname=srs, outputFormat='application/json').read()

geo = geojson.loads(json_response)
print('geo done')
flip_geojson_coordinates(geo)
print('flip done')

geo done
flip done


In [26]:
print(geo.keys())

dict_keys(['crs', 'type', 'features'])


In [27]:
print(geo['type'])

FeatureCollection


In [39]:
#This cell requires cartopy, but its not in the Jupyter environment and won't import

import cartopy.crs as ccrs
geodetic = ccrs.Geodetic(globe=ccrs.Globe(datum='WGS84'))

plt.figure(figsize=(12,12))
# Open Source Imagery from MapQuest (max zoom = 16?) [SMR2018-05-18 doesn't work]
#tiler = MapQuestOpenAerial()
# Open Street Map (max zoom = 18?)
tiler = OSM()
ax = plt.axes(projection=tiler.crs)
dx=b[2]-b[0]
dy=b[3]-b[1]
extent = (b[0]-0.1*dx,b[2]+0.1*dx,b[1]-0.1*dy,b[3]+0.1*dy)
ax.set_extent(extent, geodetic)
ax.add_image(tiler, 14)
#ax.add_geometries([polygon],ccrs.PlateCarree(),
#                          facecolor=BLUE, edgecolor=GRAY,alpha=0.5)
for p in geo.get("features", []):
    multi_poly = asShape(p.get("geometry"))
    print ('bounds from Shapely: ',multi_poly.bounds)
#    name=p['properties']['NAME']
#    print name
    ax.add_geometries(multi_poly,ccrs.PlateCarree(),
                edgecolor='black',facecolor='none',hatch='/')
#title(name)
    
gl=ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
#ax.add_feature(coast_10m,edgecolor='black')
#ax.coastlines()

ImportError: No module named 'cartopy'

In [40]:
!pip3 install geopandas
!pip3 install geojsonio

    100% |████████████████████████████████| 890kB 14.8MB/s ta 0:00:01
    100% |████████████████████████████████| 15.7MB 3.2MB/s eta 0:00:01
  Running setup.py bdist_wheel for click-plugins ... done
  Stored in directory: /home/zeppelin-v/.cache/pip/wheels/bc/86/aa/7680e54728998edc1e966a20249b920e582012ed6bdd3815c4
  Running setup.py bdist_wheel for munch ... done
  Stored in directory: /home/zeppelin-v/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built click-plugins munch
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.5/dist-packages/descartes'
Consider using the `--user` option or check the permissions.

    100% |████████████████████████████████| 143kB 8.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for geojsonio ... done
  Stored in directory: /home/zeppelin-v/.cache/pip/wheels/a9/ef/7c/7bbf228825e8717adaa84cd4b6c4ed8649b7958dd2bac45076
Successfully built geojsonio
Could no

In [41]:
import geopandas as gpd

ImportError: No module named 'geopandas'